In [0]:
# distinct()
#dropduplicate()
#window function with Row_number()
# groupby with count()


In [0]:
dbutils.fs.put("/scenario/duplicates.csv","""id,name,loc,updated_date
1,ravi,bangalore,2024-06-02
1,ravi,chenai,2024-06-03
1,ravi,hyderabad,2024-06-04
2,raj,bangalore,2024-06-01
2,raj,chennai,2024-06-05
3,raj,Hyderabad,2024-06-02
4,prasad,bangalore,2024-06-02
5,mahesh,chennai,2024-06-09
4,prasad,Hyderabad,2024-06-05""",True)

Wrote 272 bytes.
Out[1]: True

In [0]:
df = spark.read.csv("/scenario/duplicates.csv",header=True,inferSchema=True)
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- loc: string (nullable = true)
 |-- updated_date: date (nullable = true)



In [0]:
display(df)

id,name,loc,updated_date
1,ravi,bangalore,2024-06-02
1,ravi,chenai,2024-06-03
1,ravi,hyderabad,2024-06-04
2,raj,bangalore,2024-06-01
2,raj,chennai,2024-06-05
3,raj,Hyderabad,2024-06-02
4,prasad,bangalore,2024-06-02
5,mahesh,chennai,2024-06-09
4,prasad,Hyderabad,2024-06-05


In [0]:
display(df.dropDuplicates())

id,name,loc,updated_date
2,raj,chennai,2024-06-05
1,ravi,bangalore,2024-06-02
3,raj,Hyderabad,2024-06-02
1,ravi,hyderabad,2024-06-04
2,raj,bangalore,2024-06-01
1,ravi,chenai,2024-06-03
5,mahesh,chennai,2024-06-09
4,prasad,Hyderabad,2024-06-05
4,prasad,bangalore,2024-06-02


In [0]:
# dropduplicates(),distict() dont work here
# so remove duplicates on column basis, sort the data based on updated date column



display(df.dropDuplicates(["id"]))



id,name,loc,updated_date
1,ravi,bangalore,2024-06-02
2,raj,bangalore,2024-06-01
3,raj,Hyderabad,2024-06-02
4,prasad,bangalore,2024-06-02
5,mahesh,chennai,2024-06-09


In [0]:
from pyspark.sql.functions import col

display(df.orderBy(col("updated_date").desc()).dropDuplicates(["id"]))

id,name,loc,updated_date
1,ravi,hyderabad,2024-06-04
2,raj,chennai,2024-06-05
3,raj,Hyderabad,2024-06-02
4,prasad,Hyderabad,2024-06-05
5,mahesh,chennai,2024-06-09


In [0]:
# using window function with row_number()


from pyspark.sql.window import Window
from pyspark.sql.functions import *

df = df.withColumn("row_id",row_number().over(Window.partitionBy("id").orderBy(col("updated_date").desc())))

In [0]:
display(df)

id,name,loc,updated_date,row_id
1,ravi,hyderabad,2024-06-04,1
1,ravi,chenai,2024-06-03,2
1,ravi,bangalore,2024-06-02,3
2,raj,chennai,2024-06-05,1
2,raj,bangalore,2024-06-01,2
3,raj,Hyderabad,2024-06-02,1
4,prasad,Hyderabad,2024-06-05,1
4,prasad,bangalore,2024-06-02,2
5,mahesh,chennai,2024-06-09,1


In [0]:
df_unique = df.filter("row_id=1")

In [0]:
df_baddata =  df.filter("row_id>1")

In [0]:
display(df_baddata)

id,name,loc,updated_date,row_id
1,ravi,chenai,2024-06-03,2
1,ravi,bangalore,2024-06-02,3
2,raj,bangalore,2024-06-01,2
4,prasad,bangalore,2024-06-02,2


In [0]:
display(df_unique)

id,name,loc,updated_date,row_id
1,ravi,hyderabad,2024-06-04,1
2,raj,chennai,2024-06-05,1
3,raj,Hyderabad,2024-06-02,1
4,prasad,Hyderabad,2024-06-05,1
5,mahesh,chennai,2024-06-09,1
